In [38]:
import pandas as pd
import numpy as np
import gc

In [39]:
train_folder = '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/training set'
test_folder = '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/test set'
full_folder = '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/full data'

In [40]:
gc.collect()

0

## Part A. Demo Converting Text File into Numpy Array 

In [41]:
# path_file = path_folder + '/20180624paths'

In [42]:
'''
results = []
with open(path_file) as inputfile:
    for line in inputfile:
        results.append(line.strip().split('|'))
'''

"\nresults = []\nwith open(path_file) as inputfile:\n    for line in inputfile:\n        results.append(line.strip().split('|'))\n"

In [43]:
# results

In [44]:
'''
valid_results = []
for row in results:
    if ',' in row[2]:
        valid_results.append(row)
'''        

"\nvalid_results = []\nfor row in results:\n    if ',' in row[2]:\n        valid_results.append(row)\n"

In [45]:
# valid_results

As shown, among 90551 paths in file 0624, only 20706 paths got more than 1 item presented

In [46]:
'''
arr = np.array(results)
val_arr = np.array(valid_results)
arr.shape, val_arr.shape
'''

'\narr = np.array(results)\nval_arr = np.array(valid_results)\narr.shape, val_arr.shape\n'

## Part B. Process all the Path Files in Folder

In [47]:
def txt2list(txt_path):
    '''
    input:
        1. txt_path: path to the text file
    output:
        2. a list containing individual lists, each of which contains:
            userid, pathid, itemids in path
    '''
    results = []
    with open(txt_path) as inputfile:
        for line in inputfile:
            results.append(line.strip().split('|'))
            
    valid_results = []
    for row in results:
        if ',' in row[2]:
            valid_results.append(row)
            
    return valid_results

In [48]:
from os import listdir
from os.path import isfile, join
def get_paths_from_folder(folder_path):    
    return [join(folder_path, f) for f in listdir(folder_path) if isfile(join(folder_path, f))]

In [49]:
file_paths = get_paths_from_folder(test_folder)
file_paths

['/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/test set/.DS_Store',
 '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Data/paths/test set/20180628paths']

In [50]:
all_paths = []
for txt_path in file_paths:
    if not txt_path.endswith('.DS_Store'):
        all_paths = all_paths + txt2list(txt_path)
len(all_paths)    

89132

In [51]:
arr = np.array(all_paths)
arr = arr.astype(object)
arr.shape

(89132, 3)

## Part C. Sort the Paths by Lengths

In [52]:
for j in range(len(arr)):
    arr[j][0] = int(arr[j][0])
    arr[j][1] = int(arr[j][1])
    arr[j][2] = list(arr[j][2].split(','))
    for i in range(len(arr[j][2])):
        arr[j][2][i] = int(arr[j][2][i])
        
arr

array([[23571, 74, list([725950920, 1236950020])],
       [215385, 56, list([980194910, 980201005])],
       [281995, 13, list([688023002, 568288207])],
       ...,
       [76346846, 12, list([702374141, 675615375])],
       [76455160, 26,
        list([1061477831, 706672870, 1132726214, 1092471096])],
       [76530139, 0, list([682505284, 642637406])]], dtype=object)

In [53]:
arr[0]

array([23571, 74, list([725950920, 1236950020])], dtype=object)

In [54]:
# create a list containing the session lengths

sess_len_list = []
for i in range(arr.shape[0]):
    sess_len_list.append(len(arr[i][2]))
sess_len_arr = np.array(sess_len_list)
sess_len_arr = sess_len_arr.reshape([len(sess_len_list),1])

In [55]:
arr = np.concatenate((arr,sess_len_arr),axis = 1)
arr.shape

(89132, 4)

In [56]:
arr[0],arr[1]

(array([23571, 74, list([725950920, 1236950020]), 2], dtype=object),
 array([215385, 56, list([980194910, 980201005]), 2], dtype=object))

#### Remarks:

Here it is necessary to note that the four columns (0-3) of the numpy array is:

0: usedid as int

1: pathid as int

2: path as np array

3: len(path) as int


In [57]:
df = pd.DataFrame(arr)
df.columns = ['userid', 'sess_id', 'sess_path', 'sess_length']

In [58]:
df = df.sort_values(by = 'sess_length')
df = df.reset_index(drop = 'True')

#### Remarks:
Here a small trouble is encountered: the pathid got duplication, but if we apply drop duplicate, disaster can happen

In [59]:
## df.drop_duplicates(subset='sess_id', keep = 'first')

In [60]:
summary = df.groupby('sess_length').count()
summary = pd.DataFrame(summary.drop(columns=['sess_id','sess_path']))
summary = summary.reset_index()
summary.columns = ['sess_length', 'count']

In [61]:
slice_table = summary
slice_table['start_index'] = slice_table['count']
slice_table['end_index'] = slice_table['count']
slice_table

,sess_length,count,start_index,end_index
0,2,59412,59412,59412
1,3,14944,14944,14944
2,4,5870,5870,5870
3,5,3175,3175,3175
4,6,2050,2050,2050
5,7,1036,1036,1036
6,8,714,714,714
7,9,401,401,401
8,10,286,286,286
9,11,209,209,209


In [62]:
# here is the logic for create the start index + end index
for i in range(1, len(slice_table)):
    slice_table['end_index'][i] = slice_table['end_index'][i-1] + slice_table['count'][i]

for i in range(1, len(slice_table)):
    slice_table['start_index'][i] = slice_table['end_index'][i-1]

slice_table['start_index'][0] = 0

slice_table

,sess_length,count,start_index,end_index
0,2,59412,0,59412
1,3,14944,59412,74356
2,4,5870,74356,80226
3,5,3175,80226,83401
4,6,2050,83401,85451
5,7,1036,85451,86487
6,8,714,86487,87201
7,9,401,87201,87602
8,10,286,87602,87888
9,11,209,87888,88097


#### Planning A:
Here it we can wrap the counting of each session length into a dictionary, when the training batch is randomly generated, the prob of session length to be selected should be proportion to the session length count.

Therefore, the index slicing range for each session length should be presented.

#### Planning B:
Use zero padding + bucketing Method to deal with the data: with different length: the candidate lengths will be: (2,3,4,5,6,10,20,50), sessions with longer length will be discarded

#### Another Planning:
The FM user/item embedding can actually be input into another RNN, say if the item/user embedding DNE: create random embedding.

(Then is it possible to train user/item embedding at RNN???

#### Some Dictionaries Needed:

In the set of active items (items that are presenting in the session paths), the item would have a unique id in this active set

so we need to map: itemid (given) ---> unique id in active item set ---> FM pretrained embedding

so we need to map: userid (given) ---> unique id in active user set --->

## Part D. User/Item Index Mapping

In [80]:
df

,userid,sess_id,sess_path,sess_length
0,176,74,"[84386, 226475]",2
1,1249,15,"[39126, 43052]",2
2,1623,1,"[61433, 105916]",2
3,2018,3,"[201134, 212315]",2
4,2252,4,"[34272, 155290]",2
5,2996,12,"[69469, 153955]",2
6,3501,17,"[10848, 205494]",2
7,5846,176,"[138868, 39448]",2
8,5959,24,"[229471, 165813]",2
9,6823,21,"[187544, 177293]",2


In [81]:
# return the slicings, which will be used to separate different length of sequences
arr = df.values

In [82]:
arr.shape

(89132, 4)

In [83]:
type(df['sess_path'][1][1])

int

In [84]:
df.head()

,userid,sess_id,sess_path,sess_length
0,176,74,"[84386, 226475]",2
1,1249,15,"[39126, 43052]",2
2,1623,1,"[61433, 105916]",2
3,2018,3,"[201134, 212315]",2
4,2252,4,"[34272, 155290]",2


In [85]:
import time

In [86]:
print(time.ctime())
'''
# now we gonna count the active users and active items
userid_list = []
itemid_list = []
for i in range(arr.shape[0]):
    userid_list.append(arr[i][0])
    itemid_list = itemid_list + arr[i][2]
    
userid_set = set(userid_list)
itemid_set = set(itemid_list)

print('Total amount of user presented before and after dropping duplication:')
print(len(userid_list), len(userid_set))
print('Total amount of item presented before and after dropping duplication:')
print(len(itemid_list), len(itemid_set))
'''
print(time.ctime())

Mon Jul  9 14:02:27 2018
Mon Jul  9 14:02:27 2018


In [87]:
'''

userid_df = pd.DataFrame([0]+list(userid_set))
itemid_df = pd.DataFrame([0]+list(itemid_set))
userid_df.columns = ['userid']
itemid_df.columns = ['itemid']
userid_df['userid'] = pd.to_numeric(userid_df['userid'])
itemid_df['itemid'] = pd.to_numeric(itemid_df['itemid'])

userid_df = userid_df.sort_values(by = 'userid').reset_index(drop=True)
itemid_df = itemid_df.sort_values(by = 'itemid').reset_index(drop=True)
'''

"\n\nuserid_df = pd.DataFrame([0]+list(userid_set))\nitemid_df = pd.DataFrame([0]+list(itemid_set))\nuserid_df.columns = ['userid']\nitemid_df.columns = ['itemid']\nuserid_df['userid'] = pd.to_numeric(userid_df['userid'])\nitemid_df['itemid'] = pd.to_numeric(itemid_df['itemid'])\n\nuserid_df = userid_df.sort_values(by = 'userid').reset_index(drop=True)\nitemid_df = itemid_df.sort_values(by = 'itemid').reset_index(drop=True)\n"

In [88]:
pro_data_fold = '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Processed Data'

In [89]:
userid_df = pd.read_csv(pro_data_fold + '/userid_map.csv')
itemid_df = pd.read_csv(pro_data_fold + '/itemid_map.csv')

In [90]:
# below is the dictionary for mapping item/user ids in shopee db into model based id

userid_dict = dict([(userid_df['userid'][i], i) for i in range(len(userid_df))])
itemid_dict = dict([(itemid_df['itemid'][i], i) for i in range(len(itemid_df))])

In [91]:
userid_dict

{0: 0,
 10002: 1,
 10026: 2,
 10028: 3,
 10050: 4,
 10057: 5,
 10058: 6,
 10095: 7,
 10107: 8,
 10108: 9,
 10116: 10,
 10124: 11,
 10147: 12,
 10155: 13,
 10159: 14,
 10174: 15,
 10214: 16,
 10268: 17,
 10287: 18,
 10290: 19,
 10295: 20,
 10308: 21,
 10310: 22,
 10359: 23,
 10360: 24,
 10370: 25,
 10378: 26,
 10388: 27,
 10394: 28,
 10398: 29,
 10420: 30,
 10461: 31,
 10484: 32,
 10509: 33,
 10520: 34,
 10523: 35,
 10530: 36,
 10536: 37,
 10556: 38,
 10563: 39,
 11117: 40,
 11122: 41,
 11171: 42,
 11271: 43,
 11285: 44,
 11304: 45,
 11351: 46,
 11355: 47,
 11358: 48,
 11405: 49,
 11465: 50,
 11570: 51,
 11671: 52,
 11702: 53,
 11714: 54,
 11733: 55,
 11918: 56,
 11921: 57,
 12520: 58,
 12521: 59,
 12571: 60,
 12584: 61,
 12597: 62,
 12672: 63,
 12695: 64,
 12713: 65,
 12763: 66,
 12801: 67,
 12805: 68,
 12816: 69,
 13140: 70,
 13214: 71,
 13253: 72,
 13277: 73,
 13313: 74,
 13870: 75,
 13960: 76,
 14061: 77,
 14062: 78,
 14125: 79,
 14683: 80,
 14710: 81,
 14750: 82,
 14869: 83,
 14930

In [92]:
for i in range(arr.shape[0]):
    arr[i][0] = userid_dict[arr[i][0]]
    for j in range(len(arr[i][2])):
        arr[i][2][j] = itemid_dict[arr[i][2][j]]

KeyError: 176

In [ ]:
arr.shape

In [ ]:
# just to show the ids in the array has been well transferred
arr[20],arr[80000],arr[70000]

## Part D. Data Padding & Bucketing

In [93]:
Y_list = []
for i in range(arr.shape[0]):
    Y_list.append(arr[i][2][-1])
    arr[i][2] = arr[i][2][:-1]
Y_arr = np.array(Y_list)
Y_arr = Y_arr.reshape([len(Y_list),1])

arr = np.concatenate((arr,Y_arr),axis = 1)
arr.shape

(89132, 5)

In [94]:
arr

array([[176, 74, list([84386]), 2, 226475],
       [1249, 15, list([39126]), 2, 43052],
       [1623, 1, list([61433]), 2, 105916],
       ...,
       [25743, 70,
        list([123629, 131001, 134192, 135809, 137121, 138846, 140341, 156874, 180875, 186964, 206014, 207030, 206811, 216520, 219805, 205336, 69243, 144737, 186939, 63835, 126055, 126590, 207042, 207069, 208146, 165278, 155082, 144750, 144752, 203991, 45290, 64286, 49176, 128971, 48255, 84947, 123678, 126578, 71032, 189497, 80165, 131106, 69649, 140218, 171319, 144728, 94343, 153354, 144675, 178400, 48264, 138195, 48241, 202386, 144774, 215957, 212564, 140223, 231199, 184332, 195812, 183318, 164499, 138193, 237962, 94341, 152518, 83139, 229471]),
        70, 122241],
       [25743, 72,
        list([69649, 140218, 171319, 144728, 94343, 153354, 144675, 178400, 48264, 138195, 48241, 202386, 144774, 215957, 212564, 140223, 231199, 184332, 195812, 183318, 164499, 138193, 237962, 94341, 152518, 83139, 229471, 131106, 80165, 18949

In [96]:
# we would directly process the numpy array that contains our data
# the numpy array has been processed with proper mapping
# we would replace the lists with a numpy array

buckets_list = [20]
# we deduct all elements in the buckets_list by 1, to fulfill the length of path without Y item
for i in range(len(buckets_list)):
    buckets_list[i] = buckets_list[i] - 1
    

In [97]:
def pad2buckets(buckets_list, in_arr):
    '''
    Input:
        1. buckets_list: a list of ints, indicating the step lengths of data we want to generate
        2. data_arr: the half-processed data array
    Output: 
        1. pro_arr: the processed array
    '''
    data_arr = in_arr
    # cursor labels the current max length of our training data
    cursor = 0
    for i in range(data_arr.shape[0]):
        # the fourth (index 3) column is the session length of the current user path
        if len(data_arr[i][2]) > buckets_list[-1]:
            break
        # move the cursor to the correct place, 
        # by right the max_length should be bigger or equal to the length of the current row of data
        while len(data_arr[i][2]) > buckets_list[cursor]:
            cursor = cursor + 1
        
        if len(data_arr[i][2]) == buckets_list[cursor]:
            data_arr[i][2] = np.array(data_arr[i][2])
        if len(data_arr[i][2]) < buckets_list[cursor]:
            # create a temp numpy array 
            #temp_path = np.array([0 for i in range(buckets_list[cursor])])
            for j in range(buckets_list[cursor]-len(data_arr[i][2])):
                # temp_path[j] = data_arr[i][2][j]
                data_arr[i][2].append(0)
            # data_arr[i][2] = temp_path
            data_arr[i][2] = np.array(data_arr[i][2])
            data_arr[i][3] = buckets_list[cursor] + 1
    print(i)       
    # now i is at the first row we want to abandon
    return data_arr[:i]

In [98]:
pro_arr = pad2buckets(buckets_list, arr)

88808


In [99]:
pro_df = pd.DataFrame(pro_arr)
pro_df.columns = ['userid', 'pathid', 'path', 'sess_length', 'Y']

In [100]:
pro_arr


array([[176, 74,
        array([84386,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0]),
        20, 226475],
       [1249, 15,
        array([39126,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0]),
        20, 43052],
       [1623, 1,
        array([61433,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0]),
        20, 105916],
       ...,
       [18733, 377,
        array([ 38913,  87523, 159502,  87338, 131176, 134072, 136614,  87590,
       140722, 159515,  38324,  87579,  40311,  38325,  87510,  93426,
        80658,  99604, 159476]),
        20, 42775],
       [18304, 19,
        array([ 58389, 148980,  13084, 109503, 115355, 146430, 156280, 222285,
       221288,  48858, 154620,  29977,  81296,  93003, 152011

In [101]:
# derive the summary/slice table for the training data

summary = pro_df.groupby('sess_length').count()
summary = pd.DataFrame(summary.drop(columns=['pathid','path','Y']))
summary = summary.reset_index()
summary.columns = ['sess_length', 'count']

slice_table = summary
slice_table['start_index'] = slice_table['count']
slice_table['end_index'] = slice_table['count']
# here is the logic for create the start index + end index
for i in range(1, len(slice_table)):
    slice_table['end_index'][i] = slice_table['end_index'][i-1] + slice_table['count'][i]

for i in range(1, len(slice_table)):
    slice_table['start_index'][i] = slice_table['end_index'][i-1]

slice_table['start_index'][0] = 0

slice_table

,sess_length,count,start_index,end_index
0,20,88808,0,88808


### Save Essential Utilities

In [102]:
pro_data_fold = '/Users/yeqi.bai/Desktop/All in One/Recommender System Project/Session Model/Processed Data'

In [104]:
np.save(pro_data_fold+'/X_test.npy', pro_arr)

## Part E. Demo Data Generator

#### List of Utilities:
1. slice_table as df (can be generated at run time)
2. arr as numpy array: the source of training data

#### Desired Output:
1. Training item numpy array with selected session length
2. Training user numpy array
3. Training label Y

all the above are supposed to come with the same batch_size 

#### Remarks:
1. The probability of each session length to be selected as training data should be proportional to the amount of session with that length presented

#### About the GRU Model Placeholders:
1. y: label place holder: shape is num_item, generated by one hot encoding 
2. itemid (mapped): shape of (batch_size, )
3. userid (mapped): shape of (batch_size, )

#### Remarks:
1. Go check the loss function

In [4]:
pro_arr = np.load(pro_data_fold+'/X_train.npy')

In [5]:
pro_arr

array([[35863, 10, array([144506]), 2, 60286],
       [13691, 12, array([25622]), 2, 39979],
       [13731, 11, array([114933]), 2, 184904],
       ...,
       [13594, 91,
        array([ 54525,  34886,  61307, 130226, 130225,  20136, 148560, 110995,
        38294,  85047,  98436, 155296, 155293,  63525,  11480, 130398,
       149263, 111733, 111722,  40847,  89078,  14280,  27207,  33290,
        33304,   7542,   9145, 113624, 159997, 109851,  71966,   1978,
         9376,   4875, 138910, 115066, 171960,   7371,   6891,  23825,
        99161,  63378,  39967, 163618,  28446,  58308,  93562, 109705,
        61442,   8227, 166182, 163879,  41640,  22422,  96402, 160336,
        30828,  88918, 101293, 101292,  99051, 107968,  49749,  11938,
         5785,  92997,   8508,  33208,  31236,  33278,  26889,  13066,
        51365,  29541,  98993, 116958,  29290, 165525,  17498,  54285,
       155268,  51156, 101336,  11484,  96480,  11936,  32596,  78255,
        25547,  40589, 196437, 192604, 

In [11]:
df = pd.DataFrame(pro_arr)

In [12]:
df.columns = ['userid', 'pathid', 'path', 'length', 'Y']
df.head()

,userid,pathid,path,length,Y
0,35863,10,[144506],2,60286
1,13691,12,[25622],2,39979
2,13731,11,[114933],2,184904
3,13983,49,[103076],2,190034
4,17865,2,[95841],2,150550


In [18]:
df = pd.DataFrame(pro_arr)
df.columns = ['userid', 'pathid', 'path', 'length', 'Y']

print("Sample of training data: ")
print(df.head())

summary = df.groupby('length').count()
summary = pd.DataFrame(summary.drop(columns=['pathid','path']))
summary = summary.reset_index()
summary = summary.drop(['Y'],axis = 1)
summary.columns = ['length', 'count']

slice_table = summary
slice_table['start_index'] = slice_table['count']
slice_table['end_index'] = slice_table['count']
slice_table

# here is the logic for create the start index + end index
for i in range(1, len(slice_table)):
    slice_table['end_index'][i] = slice_table['end_index'][i-1] + slice_table['count'][i]

for i in range(1, len(slice_table)):
    slice_table['start_index'][i] = slice_table['end_index'][i-1]

slice_table['start_index'][0] = 0
print("Here is the slicing table of the data: ")
print(slice_table)

Sample of training data: 
  userid pathid      path length       Y
0  35863     10  [144506]      2   60286
1  13691     12   [25622]      2   39979
2  13731     11  [114933]      2  184904
3  13983     49  [103076]      2  190034
4  17865      2   [95841]      2  150550
Here is the slicing table of the data: 
   length   count  start_index  end_index
0       2  202142            0     202142
1       3   50820       202142     252962
2       4   20257       252962     273219
3       5   10362       273219     283581
4       6    5402       283581     288983
5      10    8525       288983     297508
6      20    3174       297508     300682
7      50     727       300682     301409
8     100     100       301409     301509


In [19]:
def analyze_data(pro_arr):
    '''
    Input:
        1. pro_arr as numpy array
    Output: 
        1. the slicing table of pro_arr
    '''
    df = pd.DataFrame(pro_arr)
    df.columns = ['userid', 'pathid', 'path', 'length', 'Y']

    print("Sample of training data: ")
    print(df.head())

    summary = df.groupby('length').count()
    summary = pd.DataFrame(summary.drop(columns=['pathid','path']))
    summary = summary.reset_index()
    summary = summary.drop(['Y'],axis = 1)
    summary.columns = ['length', 'count']

    slice_table = summary
    slice_table['start_index'] = slice_table['count']
    slice_table['end_index'] = slice_table['count']
    slice_table

    # here is the logic for create the start index + end index
    for i in range(1, len(slice_table)):
        slice_table['end_index'][i] = slice_table['end_index'][i-1] + slice_table['count'][i]

    for i in range(1, len(slice_table)):
        slice_table['start_index'][i] = slice_table['end_index'][i-1]

    slice_table['start_index'][0] = 0
    print("Here is the slicing table of the data: ")
    print(slice_table)
    
    return slice_table
    

In [36]:
# slice_table['end_index'][len(slice_table)-1]
m = pro_arr.shape[0]
np.random.randint(0,m)

167586

In [139]:
def next_batch(batch_size, pro_arr, slice_table):
    '''
    Input:
        1. batch_size as integer
        2. pro_arr as numpy array, with columns: 
            [0] userid as integer
            [1] pathid as integer
            [2] path as numpy array containing itemids as integers
            [3] session_length as integer = length_input + length_output = length_input + 1
            [4] label (output itemid) as integer
    Output:
        1. next_batch_userids as numpy array, with size: [batch_size, ] or [batch_size, num_time_step]
        2. next_batch_itemids as numpy array, with size: [batch_size, num_time_step]
        3. next_batch_y as numpy array, with size: [batch_size, ]
    '''
    # first step: use probability to random select a session length to generate the next batch
    m = pro_arr.shape[0]
    rand = np.random.randint(0,m)
    length = 0
    start_index = 0
    end_index = 0
    for i in range(len(slice_table)):
        if rand >= slice_table['start_index'][i] and rand < slice_table['end_index'][i]:
            length = slice_table['length'][i]
            start_index = slice_table['start_index'][i]
            end_index = slice_table['end_index'][i]
            break
    print("generating next batch with session length t_step =", length)
    next_batchids = np.random.randint(start_index, end_index, batch_size)
    next_batch_userids = pro_arr[next_batchids][:,0]
    next_batch_itemids = np.array(pro_arr[next_batchids][:,2].tolist())
    next_batch_y = pro_arr[next_batchids][:,-1]
    
    return next_batch_userids, next_batch_itemids, next_batch_y    

In [ ]:
# step 1: decide a length for the session
# we are going to take session length <= 18 
